# NoSQL: Neo4j Database Analysis
## By: Daisy Pinaroc

In [48]:
from IPython.core.magic import register_cell_magic
import shlex

In [ ]:
%env CYPHER=XXXX
%env HOST=XXXX  
%env USER=XXXX
%env PW=XXXX

In [69]:
CONNECT="$CYPHER -a $HOST -u $USER -p $PW"

In [70]:
@register_cell_magic('neo')
def neo(line, cell):
    !{CONNECT} {shlex.quote(cell)}

### Testing connectivity

In [51]:
%%neo
MATCH (n)
RETURN n LIMIT 5;

+------------------------------------+
| n                                  |
+------------------------------------+
| (:Author {name: "Tony Tobin"})     |
| (:Author {name: "Jane Hornby"})    |
| (:Author {name: "Sarah Cook"})     |
| (:Author {name: "Good Food"})      |
| (:Author {name: "Jennifer Joyce"}) |
+------------------------------------+

5 rows
ready to start consuming query after 9 ms, results consumed after another 1 ms


# Begin Neo4j work

### 1) Returns the unique node labels and the number of nodes for each. Sorts the results by the number of nodes in descending order.

In [52]:
%%neo
MATCH (n)
RETURN DISTINCT LABELS(n) as node_label, COUNT(*) as num_nodes
ORDER BY num_nodes DESC;

+----------------------------+
| node_label     | num_nodes |
+----------------------------+
| ["Recipe"]     | 11634     |
| ["Ingredient"] | 3077      |
| ["Collection"] | 1049      |
| ["Author"]     | 303       |
| ["DietType"]   | 12        |
+----------------------------+

5 rows
ready to start consuming query after 11 ms, results consumed after another 10 ms


### 2) Returns the unique relationship types and the number of relationships for each. Sorts the results by the number of relationships in descending order.

In [53]:
%%neo
MATCH ()-[r]->() 
RETURN DISTINCT TYPE(r) as relationship_type, COUNT(r) as num_relationships
ORDER BY num_relationships DESC;

+-------------------------------------------+
| relationship_type     | num_relationships |
+-------------------------------------------+
| "CONTAINS_INGREDIENT" | 106148            |
| "COLLECTION"          | 32240             |
| "DIET_TYPE"           | 14858             |
| "WROTE"               | 11634             |
+-------------------------------------------+

4 rows
ready to start consuming query after 11 ms, results consumed after another 26 ms


### 3) Finds all the recipes that contain both "chicken" and "lemon" as ingredients, but were NOT written by the author "Good Food".

In [56]:
%%neo
MATCH (a:Author)-[:WROTE]->(r:Recipe)-[:CONTAINS_INGREDIENT]->(i1:Ingredient {name: 'chicken'}), (r)-[:CONTAINS_INGREDIENT]->(i2:Ingredient {name: 'lemon'})
WHERE NOT a.name = 'Good Food' 
RETURN a.name as recipe_author, r as recipe
LIMIT 3;

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| recipe_author   | recipe                                                                                                                                                                                                                                                                                                                                             |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

### 4) Returns recipes that are either dairy-free, fall under the "Christmas dinner" collection, or is written by author John Torode.

In [83]:
%%neo
MATCH (r:Recipe),
(r)-[:DIET_TYPE]->(dt:DietType),
(r)-[:COLLECTION]->(c:Collection),
(r)<-[:WROTE]-(a:Author) 
WHERE dt.name = 'Dairy-free' or c.name = 'Christmas dinner' or a.name = 'John Torode'
RETURN r.name as recipe, r.description as recipe_description, dt.name AS diet_type, c.name AS collection, a.name AS author
LIMIT 5; 

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| recipe                                   | recipe_description                                                                                                                    | diet_type    | collection           | author        |
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| "Cranberry sauce with port & star anise" | "Star anise gives this classic sauce a really Christmassy flavour, and a contemporary update"                                         | "Low-salt"   | "Christmas dinner"   | "Tony Tobin"  |
| "Strawberry & rose sorbet"               | "A ruby-red ice

### 5) Finds recipes that are both part of the "Christmas dinner" collection and contain chocolate as an ingredient.

In [64]:
%%neo
MATCH (rp:Recipe)-[r:COLLECTION]->(c:Collection {name: 'Christmas dinner'}), (rp:Recipe)-[:CONTAINS_INGREDIENT]->(i:Ingredient {name: 'chocolate'})
RETURN rp.name as recipe, rp.description AS recipe_description, rp.skillLevel AS recipe_skill_level 
ORDER BY rp.name ASC 
LIMIT 3;

+-----------------------------------------------------------------------------------------------------------------------------------------------------------+
| recipe               | recipe_description                                                                                            | recipe_skill_level |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------+
| "Yule chocolate log" | "Mary Cadoganâ€™s festive chocolate log will go down a treat with everyone - it's a lighter recipe than most" | "More effort"      |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------+

1 row
ready to start consuming query after 90 ms, results consumed after another 5 ms


### 6) Returns recipes that DON'T contain onion OR mozzarella.

In [113]:
%%neo
MATCH (r:Recipe)
WHERE NOT (r)-[:CONTAINS_INGREDIENT]->(:Ingredient {name: 'onion'})
    OR NOT (r)-[:CONTAINS_INGREDIENT]->(:Ingredient {name: 'mozzarella'})
RETURN r as recipe
ORDER BY r.name
LIMIT 3;

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| recipe                                                                                                                                                                                                                                                                                                               |
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| (:Recipe {name: " Apricot & pistachio frangipane blondies",

### 7) Finds Vegan summer recipes written by Miriam Nice.

In [81]:
%%neo
MATCH (rp:Recipe)-[r: DIET_TYPE]->(dt:DietType {name:'Vegan'}), 
(rp)-[:COLLECTION]->(c:Collection {name: 'Summer'}), 
(rp)<-[:WROTE]-(a:Author {name: 'Miriam Nice'})
RETURN rp AS recipe, dt AS diet_type, c AS collection, a AS author
ORDER BY rp.name;

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| recipe                                                                                                                                                                                                                                                                                    | diet_type                   | collection                     | author                          |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

### 8) Returns the IDs and relationship type of the last few relationships in the dataset.

In [117]:
%%neo
MATCH ()-[r]->()
RETURN TYPE(r) as relationship_type, ID(r) as relationship_id
ORDER BY relationship_id DESC
LIMIT 5;

+-------------------------------------+
| relationship_type | relationship_id |
+-------------------------------------+
| "COLLECTION"      | 164879          |
| "COLLECTION"      | 164878          |
| "COLLECTION"      | 164877          |
| "COLLECTION"      | 164876          |
| "COLLECTION"      | 164875          |
+-------------------------------------+

5 rows
ready to start consuming query after 42 ms, results consumed after another 102 ms


### 9) Returns recipes that are in the "Dinner party starter" collection, are a "Low-fat" diet-type, and don't contain cheese as an ingredient.

In [116]:
%%neo
MATCH (r:Recipe)-[:COLLECTION]->(c:Collection {name: 'Dinner party starter'}),
      (r)-[:DIET_TYPE]->(d:DietType {name: 'Low-fat'}),
      (r)-[:CONTAINS_INGREDIENT]->(i:Ingredient)
WHERE NOT i.name = 'cheese'
MATCH (r)<-[:WROTE]-(a:Author)
RETURN DISTINCT a.name AS author_name, r.name AS recipe, d.name AS diet_type, c.name AS collection
ORDER BY author_name; 

+----------------------------------------------------------------------------------------------------+
| author_name      | recipe                                     | diet_type | collection             |
+----------------------------------------------------------------------------------------------------+
| ""               | "Melon with mint & feta"                   | "Low-fat" | "Dinner party starter" |
| "Good Food"      | "Creamy smoked salmon, leek & potato soup" | "Low-fat" | "Dinner party starter" |
| "Jennifer Joyce" | "Prawn cakes with cucumber peanut relish"  | "Low-fat" | "Dinner party starter" |
+----------------------------------------------------------------------------------------------------+

3 rows
ready to start consuming query after 177 ms, results consumed after another 15 ms


### 10) Returns the relationship IDs for recipes written by Tana Ramsay. 

In [115]:
%%neo
MATCH (a:Author {name: 'Tana Ramsay'})-[rel:WROTE]->(r:Recipe)
RETURN ID(a) as author_id, TYPE(rel) as relationship_type, ID(rel) as relationship_id;

+-------------------------------------------------+
| author_id | relationship_type | relationship_id |
+-------------------------------------------------+
| 71        | "WROTE"           | 8516            |
| 71        | "WROTE"           | 5689            |
| 71        | "WROTE"           | 6375            |
| 71        | "WROTE"           | 483             |
| 71        | "WROTE"           | 7635            |
+-------------------------------------------------+

5 rows
ready to start consuming query after 51 ms, results consumed after another 3 ms


# More Neo4j Practice

### 1) Creates a new node for each existing node label with at least one property for a signature coffee chocolate cake recipe.

In [114]:
%%neo
CREATE (:Recipe {name: "Coffee Chocolate Cake"});
CREATE (:Ingredient {name: "chocolate"});
CREATE (:Collection {name: "Birthday"});
CREATE (:Author {name: "Daisy Pinaroc"});
CREATE (:DietType {name: "Nut-free"});

0 rows
ready to start consuming query after 345 ms, results consumed after another 0 ms
Added 1 nodes, Set 1 properties, Added 1 labels
0 rows
ready to start consuming query after 25 ms, results consumed after another 0 ms
Added 1 nodes, Set 1 properties, Added 1 labels
0 rows
ready to start consuming query after 20 ms, results consumed after another 0 ms
Added 1 nodes, Set 1 properties, Added 1 labels
0 rows
ready to start consuming query after 20 ms, results consumed after another 0 ms
Added 1 nodes, Set 1 properties, Added 1 labels
0 rows
ready to start consuming query after 20 ms, results consumed after another 0 ms
Added 1 nodes, Set 1 properties, Added 1 labels


### 2) Create a new relationship for each existing relationship type:
* Adds the "Herby slashed roasties" Recipe under the "Comfort food" Collection
* Adds the "Butternut soup shots with crispy pancetta soldiers" Recipe under the "Low-sugar" Diet Type
* Includes ground cumin as an ingredient for "Sizzled lamb steaks with warm beetroot salad"
* Adds Daisy Pinaroc as an author for the "Creamy tarragon chicken & potato bake" recipe

In [118]:
%%neo
MATCH (r:Recipe {name:'Herby slashed roasties'})
MATCH (c:Collection {name: 'Comfort food'})
CREATE (r)-[:COLLECTION]->(c);

MATCH (r:Recipe {name:'Butternut soup shots with crispy pancetta soldiers'})
MATCH (dt:DietType {name:'Low-sugar'})
CREATE (r)-[:DIET_TYPE]->(dt);

MATCH (r:Recipe {name:'Sizzled lamb steaks with warm beetroot salad'})
MATCH (i:Ingredient {name:'ground cumin'})
CREATE (r)-[:CONTAINS_INGREDIENT]->(i);

MATCH (a:Author {name:'Daisy Pinaroc'})
MATCH (r:Recipe {name:'Creamy tarragon chicken & potato bake'})
CREATE (a)-[:WROTE]->(r);

0 rows
ready to start consuming query after 142 ms, results consumed after another 0 ms
Created 1 relationships
0 rows
ready to start consuming query after 59 ms, results consumed after another 0 ms
Created 1 relationships
0 rows
ready to start consuming query after 58 ms, results consumed after another 0 ms
Created 1 relationships
0 rows
ready to start consuming query after 54 ms, results consumed after another 0 ms
Created 1 relationships


### 3) Updates the properties of the "Crispy duck pancakes" Recipe node. Adds more to the recipe description and changes the skill level from "Easy" to "Medium".

In [121]:
%%neo
MATCH (r:Recipe {name: 'Crispy duck pancakes'})
SET r.description = "Don't call the Chinese takeaway for crispy duck with pancakes – our recipe is so easy to make and it's healthier, too. Serve with a classic plum sauce and syrup if that's what you prefer. Perfect for Thanksgiving!",
    r.skillLevel = "Medium"
RETURN r.name, r.description, r.skillLevel; 

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| r.name                 | r.description                                                                                                                                                                                                           | r.skillLevel |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| "Crispy duck pancakes" | "Don't call the Chinese takeaway for crispy duck with pancakes – our recipe is so easy to make and it's healthier, too. Serve with a classic plum sauce and syrup if that's what you prefer. Perf

### 4) Updating the properties of the COLLECTION relationship between the "Coconut hot chocolate" recipe and "Festive drinks" collection to have a new property called "weather". This query sets "cold" to that property.

In [126]:
%%neo
MATCH (r:Recipe {name: 'Coconut hot chocolate'}), (c:Collection {name: 'Festive drinks'})
MERGE (r)-[rel:COLLECTON]->(c)
ON MATCH SET rel.weather = 'cold' 
RETURN r.name, type(rel), rel.weather; 

+-----------------------------------------------------+
| r.name                  | type(rel)   | rel.weather |
+-----------------------------------------------------+
| "Coconut hot chocolate" | "COLLECTON" | "cold"      |
+-----------------------------------------------------+

1 row
ready to start consuming query after 21 ms, results consumed after another 31 ms
Set 1 properties


### 5) Added a new node label called "Special" to the existing Recipe node "Marinated lamb steaks with paprika roast tomatoes & butterbean smash".

In [127]:
%%neo
MATCH (r {name: 'Marinated lamb steaks with paprika roast tomatoes & butterbean smash'})
SET r:Special
RETURN r.name, labels(r) AS labels;

+------------------------------------------------------------------------------------------------+
| r.name                                                                 | labels                |
+------------------------------------------------------------------------------------------------+
| "Marinated lamb steaks with paprika roast tomatoes & butterbean smash" | ["Recipe", "Special"] |
+------------------------------------------------------------------------------------------------+

1 row
ready to start consuming query after 57 ms, results consumed after another 37 ms
Added 1 labels


### 6) Renaming the relationship type "COLLECTION" to "IS_COLLECTION".

In [130]:
%%neo
MATCH (r:Recipe)-[rel:COLLECTION]->(c:Collection)
MERGE (r)-[:IS_COLLECTION]->(c)
DELETE rel;

MATCH (r:Recipe)-[rel2:IS_COLLECTION]->(c:Collection)
RETURN r, rel2, c
LIMIT 5

0 rows
ready to start consuming query after 113 ms, results consumed after another 0 ms
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| r                                                                                                                                                                                                                                                                    | rel2             | c                                 |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

### 7) Deletes all the relationships between the author "Ping Coombes" and all the recipes they wrote.

In [131]:
%%neo
MATCH (a:Author {name: 'Ping Coombes'})-[rel:WROTE]->(r:Recipe)
DELETE rel

0 rows
ready to start consuming query after 53 ms, results consumed after another 0 ms
Deleted 8 relationships


### 8) Deleted all recipe nodes that had less than 9 ingredients and were under the "Cannelloni" collection.

In [132]:
%%neo
MATCH (c:Collection {name: 'Cannelloni'})<-[:IS_COLLECTION]-(r:Recipe)-[:CONTAINS_INGREDIENT]->(i)
WITH r, count(i) as num_ingredients
WHERE num_ingredients < 9
DETACH DELETE r; 

0 rows
ready to start consuming query after 104 ms, results consumed after another 0 ms
Deleted 2 nodes, Deleted 24 relationships


### 9) Creates a unique constraint for the "id" property for all Recipe nodes.

In [136]:
%%neo
CREATE CONSTRAINT unique_recipe_node_id_constraint IF NOT EXISTS
FOR (r:Recipe)
REQUIRE r.id IS NODE UNIQUE;

0 rows
ready to start consuming query after 6 ms, results consumed after another 0 ms


### 10) Creates a unique constraint for the relationship ID for the DIET_TYPE relationship.

In [139]:
%%neo
CREATE CONSTRAINT unique_diet_type_rel_constraint IF NOT EXISTS
FOR ()-[r:DIET_TYPE]-()
REQUIRE r.id IS RELATIONSHIP UNIQUE

0 rows
ready to start consuming query after 66 ms, results consumed after another 0 ms
Added 1 constraints
